In [268]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
sd_comp = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Preview dataset
sd_comp.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### District Summary

- Calculate the total number of schools

- Calculate the total number of students

- Calculate the total budget

- Calculate the average math score

- Calculate the average reading score

- Calculate the percentage of students with a passing math score (70 or greater)

- Calculate the percentage of students with a passing reading score (70 or greater)

- Calculate the percentage of students who passed math and reading (% Overall Passing)

- Create a dataframe to hold the above results

- Optional: give the displayed data cleaner formatting

In [302]:
#Create variables for all calculations for District Summary:
#Calculate total number of schools
school_count = len(sd_comp["school_name"].unique())
#Calculate total number of students
student_count = len(sd_comp["student_name"])
#Calculate total budget (one budget amount per school)
    #sums budget of every student in a given school
budgetperschool = sd_comp.groupby("school_name")["budget"].sum()
    #counts every student in each school
studentperschool = sd_comp.groupby("school_name")["student_name"].count()
    #calculates actual budget per school in a list
budgetbyschool = budgetperschool/studentperschool
    #sums all school budgets
budget = budgetbyschool.sum()
#calculate average math and reading scores
avgmath = sd_comp["math_score"].mean()
avgread = sd_comp["reading_score"].mean()
#calculate passing grades
    #calculate total passes per subject
passmath = sd_comp.query("math_score>=70")["math_score"].count()
passread = sd_comp.query("reading_score>=70")["reading_score"].count()
    #calculate pass percent by subject
ppercmath = passmath/student_count
ppercread = passread/student_count
    #calculate students who passed both subjects
passboth = sd_comp.query("math_score>=70 and reading_score >=70")["reading_score"].count()
    #calculate percent who passed both subjects
ppercboth = passboth/student_count
#Rename school_name column
sd_comp= sd_comp.rename(columns={"school_name": "School Name"})

In [303]:
#Create DataFrame table for District Summary
DisSumm = pd.DataFrame({"Total Schools": [school_count],
                           "Total Students": [student_count],
                           "Total Budget": budget,
                           "Average Math Score": avgmath,
                            "Average Reading Score": avgread,
                           "% Passing Math": ppercmath*100,
                           "% Passing Reading":ppercread*100,
                           "% Overall Passing": ppercboth*100})
# Perform mapping to update formatting
DisSumm["Total Students"] = DisSumm["Total Students"].map("{:,}".format)
DisSumm["Total Budget"] = DisSumm["Total Budget"].map("${:,.2f}".format)
DisSumm["Average Math Score"] = DisSumm["Average Math Score"].map("{:.2f}".format)
DisSumm["Average Reading Score"] = DisSumm["Average Reading Score"].map("{:.2f}".format)
DisSumm["% Passing Math"] = DisSumm["% Passing Math"].map("{:.2f}%".format)
DisSumm["% Passing Reading"] = DisSumm["% Passing Reading"].map("{:.2f}%".format)
DisSumm["% Overall Passing"] = DisSumm["% Overall Passing"].map("{:.2f}%".format)
# Display formatted table
DisSumm

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [305]:
## define additional variables

# identify the school type for each school
typea = sd_comp.groupby("School Name")["type"].unique()
# convert the school type into a string (from list)
typeb = typea.astype(str).str.replace('\[|\]|\'', '')
# group the average math and reading scores by school name
mathmean = sd_comp.groupby("School Name")["math_score"].mean()
readmean = sd_comp.groupby("School Name")["reading_score"].mean()
# perform a list comprehension where:
    # for each student in the dataframe, if the math score (or reading score) is >=70, assign a "1". If not, assign a "0"
mathflag = [1 if x>=70 else 0 for x in sd_comp["math_score"]]
readflag = [1 if x>=70 else 0 for x in sd_comp["reading_score"]]
# zip together the math and reading passing flags
overallzip = zip(mathflag,readflag)
# create an empty list flag that will hold a "1" if a student passed both subjects or else a "0"
overallflag = []
# review the zipped flag list and if the student passed math (1) and reading (1) (1+1=2), add a "1" to the overall flag list, or else and a "0"
for x,y in overallzip:
    if (x + y) == 2:  
        overallflag.append(1)
    else: overallflag.append(0)
# create a new dataframe that will hold the three flags created called overall_table as a copy of the original dataframe
overall_table = sd_comp.copy()
# add three new columns
overall_table["Pass_Math"]= mathflag
overall_table["Pass_Reading"]= readflag
overall_table["Pass_Both"]= overallflag
# since each student who passed mth/reading/both will have a 1 in the respective column, group by school name and sum
passmsch = overall_table.groupby("School Name")["Pass_Math"].sum()
passrsch = overall_table.groupby("School Name")["Pass_Reading"].sum()
passosch = overall_table.groupby("School Name")["Pass_Both"].sum()
# displays new dataframe for reference
overall_table.head()




,Student ID,student_name,gender,grade,School Name,reading_score,math_score,School ID,type,size,budget,Pass_Math,Pass_Reading,Pass_Both
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,1,0,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,0,1,0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,0,1,0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,0,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,1,1,1


In [306]:
#Create DataFrame table for School Summary
SchoolSumm = pd.DataFrame({ "School Type": typeb,
                            "Total Students": studentperschool,
                            "Total School Budget": budgetbyschool,
                            "Per Student Budget": budgetbyschool/studentperschool,
                            "Average Math Score": mathmean,
                            "Average Reading Score":readmean,
                            "% Passing Math": passmsch/studentperschool, 
                            "% Passing Reading":passrsch/studentperschool,
                            "% Overall Passing":passosch/studentperschool
})
#Since SchoolSumm will be used in other calculations, create a copy of the DataFrame to map for formatting
SchoolSummFormatted = SchoolSumm.copy()
SchoolSummFormatted["Total Students"] = SchoolSummFormatted["Total Students"].map("{:,}".format)
SchoolSummFormatted["Total School Budget"] = SchoolSummFormatted["Total School Budget"].map("${:,.2f}".format)
SchoolSummFormatted["Per Student Budget"] = SchoolSummFormatted["Per Student Budget"].map("${:,.2f}".format)
SchoolSummFormatted["Average Math Score"] = SchoolSummFormatted["Average Math Score"].map("{:.2f}".format)
SchoolSummFormatted["Average Reading Score"] = SchoolSummFormatted["Average Reading Score"].map("{:.2f}".format)
SchoolSummFormatted["% Passing Math"] = SchoolSummFormatted["% Passing Math"].map("{:.2f}%".format)
SchoolSummFormatted["% Passing Reading"] = SchoolSummFormatted["% Passing Reading"].map("{:.2f}%".format)
SchoolSummFormatted["% Overall Passing"] = SchoolSummFormatted["% Overall Passing"].map("{:.2f}%".format)

# Display formatted table
SchoolSummFormatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,0.67%,0.82%,0.55%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,0.94%,0.97%,0.91%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,0.66%,0.81%,0.53%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,0.68%,0.79%,0.54%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,0.93%,0.97%,0.91%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,0.67%,0.81%,0.54%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,0.93%,0.96%,0.89%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,0.66%,0.81%,0.54%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,0.66%,0.81%,0.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,0.95%,0.96%,0.91%


### Top Performing Schools (By % Overall Passing)

* Create a table that highlights the top 5 performing schools based on % Overall Passing. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [307]:
# Copy the overall_table above
top_schools = SchoolSumm.sort_values(by=['% Overall Passing'], ascending = False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


### Bottom Performing Schools (By % Overall Passing)

* Create a table that highlights the bottom 5 performing schools based on % Overall Passing. Include all of the same metrics as above.

In [308]:
worst_schools = SchoolSumm.sort_values(by=['% Overall Passing'], ascending = True)
worst_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


### Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [310]:
columnsa = ["9th","10th","11th","12th"]
mathbygrade = pd.pivot_table(sd_comp,index=["School Name"],values=["math_score"],columns=["grade"])

mathbygrade.head(15)

math_score                                 
grade                       10th       11th       12th        9th
School Name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

### Reading Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [311]:
bygrades = sd_comp.loc[:,["School Name","grade","reading_score"]]
bygrades["grade"] = bygrades["grade"].replace("th","",regex=True).astype('float')
bins = [0,9,10,11,12]
cats = ["9th","10th","11th","12th"]
bygrades['Grade Level'] = pd.cut(bygrades["grade"],bins,labels=cats)
writegrades = bygrades.drop(columns=["grade"])
writegrades = writegrades.set_index("School Name")
writegrades = writegrades.groupby(["School Name","Grade Level"])["reading_score"].mean().unstack()

tester = pd.DataFrame(writegrades)
tester

#print(twethr)
#bygrades



Grade Level,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


### Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [312]:
#test to see the total range of spend by school per student is
print(min(budgetbyschool/studentperschool),max(budgetbyschool/studentperschool))
scorebyspend = SchoolSumm.copy()
binss = [0,599,615,630,660]
catss = ["<$599","$600-615","$616-630",">$631"]
scorebyspend['Spending Ranges (Per Student)'] = pd.cut(scorebyspend["Per Student Budget"],binss,labels=catss)
scorebyspend = scorebyspend.groupby('Spending Ranges (Per Student)').mean()
scorebyspend.drop(columns=["Total Students","Total School Budget","Per Student Budget"])



578.0 655.0


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$599,83.455399,83.933814,0.934601,0.966109,0.903695
$600-615,83.599686,83.885211,0.942309,0.959003,0.902163
$616-630,80.199966,82.425360,0.800362,0.895361,0.726209
>$631,77.866721,81.368774,0.703473,0.829956,0.588587


### Scores by School Size

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [313]:
print(min(SchoolSumm["Total Students"]),max(SchoolSumm["Total Students"]))
scorebysize = SchoolSumm.copy()
binsz = [0,999,1999,5000]
catsz = ["Small (<1,000)","Medium (1000-2000)","Large(2000-5000)"]
scorebysize['School Size'] = pd.cut(scorebysize["Total Students"],binsz,labels=catsz)
scorebysize = scorebysize.groupby('School Size').mean()
scorebysize.drop(columns=["Total Students","Total School Budget","Per Student Budget"])

427 4976


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
"Small (<1,000)",83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large(2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


### Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [314]:
scorebytype = SchoolSumm.copy()
scorebytype = scorebytype.groupby('School Type').mean()
scorebytype.drop(columns=["Total Students","Total School Budget","Per Student Budget"])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
